Après le rdv du 05/02, on en sort qu'on veut garder: 
* Une vitesse constante donnée au mur
* Toujours vouloir faire le mouvement le plus rapidement possible. 

On veut changer/ ajouter: 
* On veut rendre le probleme plus complexe en trackant la position du bout de la grue pour faire ce qu'on veut faire. On veut donner la range dans laquelle on peut se trouver pour que le bac suive le mouvement voulu. 
* On veut donner comme limite de glissement le cone de friction du mur qui pivote (A chaque time-step). 


Qu'est-ce qu'on veut minimiser ? 
* 

In [2]:
import sys
print(sys.executable)


c:\Users\loicm\Documents\Project\These\memoire_makette\.venv\Scripts\python.exe


In [38]:
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

a = 0.300   # m    
b = 0.400   # m
m = 7.0     # kg
mu = 0.3
Fmax = 40.0 # N
Icom = (m/12)*(a*a + b*b)




opti = ca.Opti()


#Position mur
phi0 = 0.0
phiT = +np.pi/2
omega = (phiT-phi0)/6
tgrid = np.linspace(0.0, 6, 121)
phi = phi0 + omega * tgrid
phi = np.minimum(phi0 + omega * tgrid, phiT)



N = 100
tf = opti.variable()
opti.subject_to(tf >= 0.1) 
dt = tf / N



pivot = ca.DM([0.0, 0.0]) 
r_CA = ca.DM([-a/2, -b/2])
r_CB = ca.DM([-a/2, +b/2]) 

    



def RM(theta):
    c, s = ca.cos(theta), ca.sin(theta)
    return ca.vertcat(
        ca.horzcat(c, -s),
        ca.horzcat(s,  c)
    )

def wall_vectors(phi):
    t = ca.vertcat(-ca.sin(phi), ca.cos(phi))  # tangent
    n = ca.vertcat( ca.cos(phi), ca.sin(phi))  # normal
    return t, n


# States
xC = opti.variable(N+1)
yC = opti.variable(N+1)
th = opti.variable(N+1)
vx = opti.variable(N+1)
vy = opti.variable(N+1)
om = opti.variable(N+1)


# Input 1
Fcx = opti.variable(N)
Fcy = opti.variable(N)


# # Contact forces of the wall at only one point
# lam_n = opti.variable(N)
# lam_t = opti.variable(N)

# Contact lambdas at wall 1
lam_nA1 = opti.variable(N)
lam_tA1 = opti.variable(N)
lam_nB1 = opti.variable(N)
lam_tB1 = opti.variable(N)

#wall 2
lam_nA2 = opti.variable(N)
lam_tA2 = opti.variable(N)
lam_nB2 = opti.variable(N)
lam_tB2 = opti.variable(N)


opti.subject_to(lam_nA1 >= 0)
opti.subject_to(lam_nB1 >= 0)
opti.subject_to(lam_nA2 >= 0)
opti.subject_to(lam_nB2 >= 0)

t_w1 = ca.vertcat(1.0,0.0) 
n_w1 = ca.vertcat(0.0,1.0)







for k in range(N):
    
    tk = k*dt
    phi_k = phi0 + omega*tk
    opti.subject_to(phi_k <= phiT)
    
    
    t_w2, n_w2 = wall_vectors(phi_k)

    #Position of A at K 
    p_C_k = ca.vertcat(xC[k], yC[k])
    Rk  = RM(th[k])
    
    p_A_k = p_C_k + Rk @ r_CA
    p_B_k = p_C_k + Rk @ r_CB

    v_A_k = ca.vertcat(vx[k] - om[k] * rA_w[1], vy[k] + om[k] * rA_w[0])
    v_B_k = ca.vertcat(vx[k] - om[k] * rB_w[1], vy[k] + om[k] * rB_w[0])


    #No penetration with the fixed wall 1; [1] = y
    opti.subject_to(p_A_k[1] >= 0)
    opti.subject_to(p_B_k[1] >= 0)

    #No penetration with the pivoting wall 2;
    gA2 = n_w2.T @ (p_A_k - pivot)
    gB2 = n_w2.T @ (p_B_k - pivot)
    opti.subject_to(gA2 >= 0)
    opti.subject_to(gB2 >= 0)


    opti.subject_to(p_A_k[1] * lam_nA1[k] <= 1e-4)
    opti.subject_to(p_B_k[1] * lam_nB1[k] <= 1e-4)
    opti.subject_to(gA2 * lam_nA2[k] <= 1e-4)
    opti.subject_to(gB2 * lam_nB2[k] <= 1e-4)

    #Dynacmics
    #Sum of forces
    Fc = ca.vertcat(Fcx[k], Fcy[k])

    opti.subject_to(lam_tA1[k] <=  mu * lam_nA1[k])
    opti.subject_to(lam_tA1[k] >= -mu * lam_nA1[k])
    opti.subject_to(lam_tA2[k] <=  mu * lam_nA2[k])
    opti.subject_to(lam_tA2[k] >= -mu * lam_nA2[k])
    
    opti.subject_to(lam_tB1[k] <=  mu * lam_nB1[k])
    opti.subject_to(lam_tB1[k] >= -mu * lam_nB1[k])
    opti.subject_to(lam_tB2[k] <=  mu * lam_nB2[k])
    opti.subject_to(lam_tB2[k] >= -mu * lam_nB2[k])

    fA1 = lam_nA1[k]*n_w1 + lam_tA1[k]*t_w1
    fB1 = lam_nB1[k]*n_w1 + lam_tB1[k]*t_w1
    fA2 = lam_nA2[k]*n_w2 + lam_tA2[k]*t_w2
    fB2 = lam_nB2[k]*n_w2 + lam_tB2[k]*t_w2
    
    opti.subject_to(Fcx[k]**2 + Fcy[k]**2 <= Fmax**2)
    fA = fA1 + fA2
    fB = fB1 + fB2
    
    Fr = fA1 + fB1 + fA2 + fB2 + Fc

    
    axk = (Fr[0]) / m
    ayk = (Fr[1]) / m
    
    opti.subject_to(xC[k+1] == xC[k] + dt * vx[k])
    opti.subject_to(yC[k+1] == yC[k] + dt * vy[k])
    opti.subject_to(vx[k+1] == vx[k] + dt * axk)
    opti.subject_to(vy[k+1] == vy[k] + dt * ayk)


    #Rotation dynamics
    rA_world = Rk @ r_CA
    rB_world = Rk @ r_CB
    tauA = rA_world[0] * fA[1] - rA_world[1] * fA[0]
    tauB = rB_world[0] * fB[1] - rB_world[1] * fB[0]
    alphak = (tauA + tauB) / Icom

    opti.subject_to(th[k+1] == th[k] + dt * om[k])
    opti.subject_to(om[k+1] == om[k] + dt * alphak)

opti.subject_to(xC[0] == 0.2)
opti.subject_to(yC[0] == 0.3)
opti.subject_to(th[0] == 0.0)
opti.subject_to(vx[0] == 0.0)
opti.subject_to(vy[0] == 0.0)
opti.subject_to(om[0] == 0.0)

#finale pose
# p_C_N = ca.vertcat(xC[N], yC[N])
# RN  = RM(th[N])
# p_B_N = p_C_N + RN @ r_CB

# opti.subject_to(th[N] == np.pi/2)
# opti.subject_to(p_B_N[1] == 0)
# opti.subject_to(p_B_N[0] >= 0) 


xC_target = 0.30
yC_target = 0.25
tol_pos   = 1e-3
opti.subject_to(xC[N] >= xC_target - tol_pos)
opti.subject_to(xC[N] <= xC_target + tol_pos)
opti.subject_to(yC[N] >= yC_target - tol_pos)
opti.subject_to(yC[N] <= yC_target + tol_pos)


tol_th = 1e-3
opti.subject_to(th[N] >= np.pi/2 - tol_th)
opti.subject_to(th[N] <= np.pi/2 + tol_th)


all_lams = ca.vertcat(
    lam_nA1, lam_tA1, lam_nB1, lam_tB1,
    lam_nA2, lam_tA2, lam_nB2, lam_tB2
)


opti.subject_to(tf <= 15.0)  
opti.subject_to(xC >= -1.0)
opti.subject_to(xC <=  2.0)
opti.subject_to(yC >=  0.10)
opti.subject_to(yC <=  2.0)


opti.set_initial(xC, np.linspace(0.1, 0.2, N+1))
opti.set_initial(yC, np.linspace(0.15, 0.25, N+1))
opti.set_initial(Fcx, 2.5)
opti.set_initial(Fcy, 0.0)
opti.set_initial(tf, 5.0)






wL = 1e-5
opti.minimize(
    tf
    + wL * ca.sumsqr(all_lams)
)


opti.solver("ipopt", {"expand": True}, {"max_iter": 2000})

sol = opti.solve()

xC_sol = sol.value(xC)
yC_sol = sol.value(yC)
th_sol = sol.value(th)
vx_sol = sol.value(vx)
vy_sol = sol.value(vy)
om_sol = sol.value(om)

Fcx_sol = sol.value(Fcx)
Fcy_sol = sol.value(Fcy)
tf_sol  = sol.value(tf)
dt_sol  = tf_sol / N

t_sol = np.linspace(0.0, tf_sol, N+1)

print("tf =", tf_sol, "s")


plt.figure()
plt.plot(xC_sol, yC_sol)
plt.axis("equal")
plt.xlabel("xC")
plt.ylabel("yC")
plt.title("Trajectoire du COM")
plt.grid(True)

plt.figure()
plt.plot(t_sol[:-1], Fcx_sol, label="Fcx")
plt.plot(t_sol[:-1], Fcy_sol, label="Fcy")
plt.xlabel("t (s)")
plt.ylabel("Force (N)")
plt.title("Forces de commande")
plt.legend()
plt.grid(True)

plt.show()




RuntimeError: Error in Opti::subject_to [OptiNode] at .../casadi/core/optistack.cpp:126:
.../casadi/core/optistack_internal.cpp:1205: Assertion "!g.is_constant()" failed:
You passed a constant to `subject_to`. You need a symbol to form a constraint.

In [47]:
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

a = 0.300   # m    
b = 0.400   # m
m = 7.0     # kg
mu = 0.3
Fmax = 100.0 # N
Icom = (m/12)*(a*a + b*b)

#Position mur
phi0 = np.pi/2
phiT = +np.pi
T_mur = 6.0 
omega_mur_fixe = (phiT - phi0) / T_mur
tgrid = np.linspace(0.0, 6, 60)
N = 100

pivot = ca.DM([0.0, 0.0]) 
r_CA = ca.DM([-a/2, -b/2])
r_CB = ca.DM([-a/2, +b/2]) 

def RM(theta):
    c, s = ca.cos(theta), ca.sin(theta)
    return ca.vertcat(
        ca.horzcat(c, -s),
        ca.horzcat(s,  c)
    )

def wall_vectors(phi):
    t = ca.vertcat(-ca.sin(phi), ca.cos(phi))  # tangent
    n = ca.vertcat( ca.cos(phi), ca.sin(phi))  # normal
    return t, n


opti = ca.Opti()

# Variables (Redéfinition propre)
tf = opti.variable(); opti.subject_to(tf >= 0.1); dt = tf / N
xC = opti.variable(N+1); yC = opti.variable(N+1); th = opti.variable(N+1)
vx = opti.variable(N+1); vy = opti.variable(N+1); om = opti.variable(N+1)
Fcx = opti.variable(N); Fcy = opti.variable(N)

# Lambdas (Forces de contact)
lam_nA1 = opti.variable(N); lam_tA1 = opti.variable(N)
lam_nB1 = opti.variable(N); lam_tB1 = opti.variable(N)
lam_nA2 = opti.variable(N); lam_tA2 = opti.variable(N)
lam_nB2 = opti.variable(N); lam_tB2 = opti.variable(N)

# Paramètres
M_big = 1000.0  

eps_contact = -1e-6  

opti.subject_to(lam_nA1 >= eps_contact)
opti.subject_to(lam_nB1 >= eps_contact)
opti.subject_to(lam_nA2 >= eps_contact)
opti.subject_to(lam_nB2 >= eps_contact)

t_w1 = ca.vertcat(1.0,0.0) 
n_w1 = ca.vertcat(0.0,1.0)

phi_seuil = 2 * np.pi / 3

for k in range(N):
    tk = k * dt
    # Position du mur à l'instant k
    phi_k = phi0 + omega_mur_fixe * tk
    phi_k = ca.if_else(phi_k > phiT, phiT, phi_k)
    
    # Indicateur : Sommes-nous encore dans la phase "verrouillée" ?
    is_locked = ca.if_else(phi_k <= phi_seuil, 1.0, 0.0)
    
    # --- 1. CONTRAINTE DE POSITION (Verrouillage) ---
    p_C_k = ca.vertcat(xC[k], yC[k])
    Rk = RM(th[k])
    p_B_k = p_C_k + Rk @ r_CB
    
    # Le point B doit être EXACTEMENT sur le pivot (ou à une distance fixe)
    # On force la distance au mur à être nulle
    t_w2, n_w2 = wall_vectors(phi_k)
    gB2 = n_w2.T @ (p_B_k - pivot)
    
    # Si is_locked == 1, gB2 doit être 0. Sinon, gB2 >= 0
    opti.subject_to(gB2 >= 0)
    opti.subject_to(gB2 <= 0.001 + M_big * (1 - is_locked))
    
    # --- 2. CONTRAINTE D'ANGLE (Verrouillage) ---
    # Le bloc doit être parfaitement aligné avec le mur
    target_th = phi_k - np.pi/2
    opti.subject_to(th[k] - target_th <= 0.01 + M_big * (1 - is_locked))
    opti.subject_to(th[k] - target_th >= -0.01 - M_big * (1 - is_locked))

    
    

    #Dynacmics
    #Sum of forces
    Fc = ca.vertcat(Fcx[k], Fcy[k])

    opti.subject_to(lam_tA1[k] <=  mu * lam_nA1[k])
    opti.subject_to(lam_tA1[k] >= -mu * lam_nA1[k])
    opti.subject_to(lam_tA2[k] <=  mu * lam_nA2[k])
    opti.subject_to(lam_tA2[k] >= -mu * lam_nA2[k])
    
    opti.subject_to(lam_tB1[k] <=  mu * lam_nB1[k])
    opti.subject_to(lam_tB1[k] >= -mu * lam_nB1[k])
    opti.subject_to(lam_tB2[k] <=  mu * lam_nB2[k])
    opti.subject_to(lam_tB2[k] >= -mu * lam_nB2[k])

    fA1 = lam_nA1[k]*n_w1 + lam_tA1[k]*t_w1
    fB1 = lam_nB1[k]*n_w1 + lam_tB1[k]*t_w1
    fA2 = lam_nA2[k]*n_w2 + lam_tA2[k]*t_w2
    fB2 = lam_nB2[k]*n_w2 + lam_tB2[k]*t_w2
    
    opti.subject_to(Fcx[k]**2 + Fcy[k]**2 <= Fmax**2)
    fA = fA1 + fA2
    fB = fB1 + fB2
    
    Fr = fA1 + fB1 + fA2 + fB2 + Fc

    
    axk = (Fr[0]) / m
    ayk = (Fr[1]) / m
    
    opti.subject_to(xC[k+1] == xC[k] + dt * vx[k])
    opti.subject_to(yC[k+1] == yC[k] + dt * vy[k])
    opti.subject_to(vx[k+1] == vx[k] + dt * axk)
    opti.subject_to(vy[k+1] == vy[k] + dt * ayk)


    #Rotation dynamics
    rA_world = Rk @ r_CA
    rB_world = Rk @ r_CB
    tauA = rA_world[0] * fA[1] - rA_world[1] * fA[0]
    tauB = rB_world[0] * fB[1] - rB_world[1] * fB[0]
    alphak = (tauA + tauB) / Icom

    opti.subject_to(th[k+1] == th[k] + dt * om[k])
    opti.subject_to(om[k+1] == om[k] + dt * alphak)

    

opti.subject_to(xC[0] == 0.15)
opti.subject_to(yC[0] == 0.2)
opti.subject_to(th[0] == np.pi/2)
opti.subject_to(vx[0] == 0.0)
opti.subject_to(vy[0] == 0.0)
opti.subject_to(om[0] == 0.0)

#finale pose
# p_C_N = ca.vertcat(xC[N], yC[N])
# RN  = RM(th[N])
# p_B_N = p_C_N + RN @ r_CB

# opti.subject_to(th[N] == np.pi/2)
# opti.subject_to(p_B_N[1] == 0)
# opti.subject_to(p_B_N[0] >= 0) 


xC_target = 0.20
yC_target = 0.15
tol_pos   = 0.01
opti.subject_to(xC[N] >= xC_target - tol_pos)
opti.subject_to(xC[N] <= xC_target + tol_pos)
opti.subject_to(yC[N] >= yC_target - tol_pos)
opti.subject_to(yC[N] <= yC_target + tol_pos)


tol_th = 1e-3
opti.subject_to(th[N] >= np.pi/2 - tol_th)
opti.subject_to(th[N] <= np.pi/2 + tol_th)


all_lams = ca.vertcat(
    lam_nA1, lam_tA1, lam_nB1, lam_tB1,
    lam_nA2, lam_tA2, lam_nB2, lam_tB2
)


opti.subject_to(tf <= 6.0)  
opti.subject_to(xC >= -1.0)
opti.subject_to(xC <=  1.0)
opti.subject_to(yC >=  0.10)
opti.subject_to(yC <=  1.5)


# opti.set_initial(xC, np.linspace(0.1, 0.2, N+1))
# opti.set_initial(yC, np.linspace(0.2, 0.23, N+1))
opti.set_initial(Fcx, 2.5)
opti.set_initial(Fcy, 0.0)
opti.set_initial(tf, 3.0)
opti.set_initial(th, np.pi/2) 




wL = 1e-5
opti.minimize(
    tf
    + wL * ca.sumsqr(all_lams)
)


opti.solver("ipopt", {"expand": True}, {"max_iter": 2000})

####################
try:
    sol = opti.solve()
except RuntimeError as e:
    print("Problème rencontré lors de la résolution :", e)
    print("Analyse des contraintes violées :")
    opti.debug.show_infeasibilities()
    raise





sol = opti.solve()

xC_sol = sol.value(xC)
yC_sol = sol.value(yC)
th_sol = sol.value(th)
vx_sol = sol.value(vx)
vy_sol = sol.value(vy)
om_sol = sol.value(om)

Fcx_sol = sol.value(Fcx)
Fcy_sol = sol.value(Fcy)
tf_sol  = sol.value(tf)
dt_sol  = tf_sol / N

t_sol = np.linspace(0.0, tf_sol, N+1)

print("tf =", tf_sol, "s")


plt.figure()
plt.plot(xC_sol, yC_sol)
plt.axis("equal")
plt.xlabel("xC")
plt.ylabel("yC")
plt.title("Trajectoire du COM")
plt.grid(True)

plt.figure()
plt.plot(t_sol[:-1], Fcx_sol, label="Fcx")
plt.plot(t_sol[:-1], Fcy_sol, label="Fcy")
plt.xlabel("t (s)")
plt.ylabel("Force (N)")
plt.title("Forces de commande")
plt.legend()
plt.grid(True)

plt.show()


# --- ANIMATION ---
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-0.5, 1.0)
ax.set_ylim(-0.2, 1.2)
ax.set_aspect('equal')
ax.grid(True)

# Éléments graphiques
wall1, = ax.plot([-1, 1], [0, 0], 'k-', lw=4, label="Mur fixe (Sol)")
wall2, = ax.plot([], [], 'r-', lw=4, label="Mur pivotant")
patch = plt.Rectangle((0, 0), a, b, color='skyblue', alpha=0.7, ec='black')
ax.add_patch(patch)
com_dot, = ax.plot([], [], 'ro', markersize=2, alpha=0.5)

def init():
    wall2.set_data([], [])
    com_dot.set_data([], [])
    return patch, wall2, com_dot

def update(k):
    # Position du bloc
    x, y, angle = xC_sol[k], yC_sol[k], th_sol[k]
    
    # Matrice de rotation pour calculer le coin bas-gauche
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle),  np.cos(angle)]])
    
    # On replace le rectangle par son coin bas-gauche (relativement au COM)
    # Le rectangle de Matplotlib tourne autour de son point d'ancrage (xy)
    corner_bg = np.array([x, y]) + R @ np.array([-a/2, -b/2])
    
    patch.set_xy(corner_bg)
    patch.angle = np.degrees(angle)
    
    # Position du mur à l'instant t_k
    tk = k * dt_sol
    phi_k = phi0 + omega_mur_fixe * tk
    if phi_k > phiT: phi_k = phiT
    
    # Dessin du mur pivotant
    L_wall = 1.2
    wall_end_x = L_wall * np.cos(phi_k)
    wall_end_y = L_wall * np.sin(phi_k)
    wall2.set_data([0, wall_end_x], [0, wall_end_y])
    
    # Historique du COM
    com_dot.set_data(xC_sol[:k], yC_sol[:k])
    
    return patch, wall2, com_dot

# Création de l'animation
ani = FuncAnimation(fig, update, frames=N+1, init_func=init, 
                    interval=(tf_sol/N)*1000, blit=True)

# --- SAUVEGARDE EN GIF ---
filename = "simulation_bloc.gif"
print(f"Enregistrement du GIF sous le nom : {filename}...")

# fps = frames par seconde (N / tf_sol)
ani.save(filename, writer='pillow', fps=int(N/tf_sol))

print("Enregistrement terminé !")
plt.legend()
plt.show()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     4406
Number of nonzeros in inequality constraint Jacobian.:     3808
Number of nonzeros in Lagrangian Hessian.............:     3499

Total number of variables............................:     1607
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      606
Total number of inequality constraints...............:     2112
        inequality constraints with only lower bounds:      707
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     1405

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.0000000e+00 1.56e+00 9.61e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:217:
.../casadi/core/optistack_internal.cpp:1338: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Maximum_Iterations_Exceeded'